In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gzip
import time

In [ ]:
dataset='z_center_p001'
npc=300
z=np.load('/oak/stanford/groups/mrivas/projects/degas-risk/datasets/train/tsvd/all_'+dataset+'_20190805_500PCs.npz')
with gzip.open('/oak/stanford/groups/mrivas/private_data/ukbb/'+
               'variant_filtering/variant_filter_table.old.tsv.gz', 'r') as f:
    var2gene={line.split('\t')[4]:line.split('\t')[12].split('|')[3] for line in f if '|' in line.split('\t')[12]}
var2gene={k:v if v else 'none' for k,v in var2gene.items()}
varcont=pd.DataFrame(z['U'][:,:npc], columns=['PC{}'.format(i+1) for i in range(npc)])
varcont.index=pd.MultiIndex.from_tuples([(var2gene[v] if v in var2gene else 'none',v) for v in z['label_var']])

In [ ]:
patterns = ('-', '+', 'x', '\\', '*', '.')
for pc in range(npc):
    print('PC'+str(pc+1))
    t=time.time()
    plt.figure(figsize=(5,5), dpi=150)
    gene2cont=dict(map(lambda g:(g,(varcont.xs(g, axis=0, level=0).iloc[:,pc]**2).sum()),
                       varcont.index.get_level_values(0)))
    print('done summing')
    base=0
    for ix,(gene,c2) in enumerate(sorted(gene2cont.items(), key=lambda x:(x[0]=='none')-x[1])):
        if c2/(varcont.iloc[:,pc]**2).sum() > 1e-3 and ix < 25:
            plt.bar(list(range(3)),
                    [c2/(varcont.iloc[:,pc]**2).sum(),0,0], 
                    bottom=[base,0,0],
                    label=gene,
                    hatch=patterns[(ix + 4) % len(patterns) ]
                   )
        else:
            plt.bar(list(range(3)), [1-base,0,0], bottom=base,  color='grey')
            plt.xticks(list(range(3)), ['PC{}'.format(pc+1),'',''])
            plt.ylabel('Gene contribution score')
            plt.legend(fontsize=6, loc='upper left', bbox_to_anchor=(1, 1))
            break
        base+=c2/(varcont.iloc[:,pc]**2).sum()
    plt.show()
    print(time.time()-t)

PC1
